In [ ]:
import pandas as pd
import psycopg2

with open('/Users/shreya/Documents/ernie_password.txt') as f:
    ernie_password = f.readline()

conn=psycopg2.connect(database="ernie",user="shreya",host="localhost",password=ernie_password)
conn.set_client_encoding('UTF8')
conn.autocommit=True
curs=conn.cursor()

# Set schema
schema = "theta_plus"
set_schema = "SET SEARCH_PATH TO " + schema + ";"   
curs.execute(set_schema)


weights = ['ncf', 'now', 'sf'] 
inflation = ['20', '30', '40', '60']

# all_data_table = 'dc_bc_merged'


for name in weights:
    for val in inflation:
        
        cluster_table = name + '_' + str(val) + '_ids'

        print("Querying: ", cluster_table)

        query1 = "select dbm.cited_1, dbm.cited_2, clt1.cluster as cited_1_cluster, clt2.cluster as cited_2_cluster from dc_bc_merged as dbm join " + cluster_table + " as clt1 on dbm.cited_1 = clt1.scp join " + cluster_table + " as clt2 on dbm.cited_2 = clt2.scp"
        query2 = "select cluster, count(cluster) as cluster_counts from " + cluster_table + " group by cluster order by cluster;"
        
        conductance_data = pd.read_sql(query1, conn)
        counts_data = pd.read_sql(query2, conn)
        
        x1 = conductance_data[conductance_data.cited_2_cluster != conductance_data.cited_1_cluster][['cited_1', 'cited_1_cluster']].groupby('cited_1_cluster', as_index=False).agg('count').rename(columns = {'cited_1': 'ext_out'})
        x2 = conductance_data[conductance_data.cited_2_cluster != conductance_data.cited_1_cluster][['cited_2', 'cited_2_cluster']].groupby('cited_2_cluster', as_index=False).agg('count').rename(columns = {'cited_2': 'ext_in'})
        x3 = conductance_data[conductance_data.cited_2_cluster == conductance_data.cited_1_cluster][['cited_1', 'cited_2_cluster']].groupby('cited_2_cluster', as_index=False).agg('count').rename(columns = {'cited_1': 'int_edges'})
        x1_clusters = counts_data.merge(x1, left_on = 'cluster', right_on = 'cited_1_cluster', how = 'left')[['cluster', 'ext_out']]
        x1_clusters = x1_clusters.fillna(0)
        x2_clusters = counts_data.merge(x2, left_on = 'cluster', right_on = 'cited_2_cluster', how = 'left')[['cluster', 'ext_in']]
        x2_clusters = x2_clusters.fillna(0)
        x3_clusters = counts_data.merge(x3, left_on = 'cluster', right_on = 'cited_2_cluster', how = 'left')[['cluster', 'int_edges', 'cluster_counts']]
        x3_clusters = x3_clusters.fillna(0)
        x4 = x1_clusters.merge(x2_clusters, left_on='cluster', right_on='cluster', how = 'inner')
        x5 = x4.merge(x3_clusters, left_on='cluster', right_on='cluster')
        x5['boundary'] = x5['ext_in'] + x5['ext_out']
        x5['volume'] = x5['ext_in'] + x5['ext_out'] + 2*x5['int_edges']
        x5['two_m'] = conductance_data.shape[0]*2
        x5['alt_denom'] = x5['two_m'] - x5['volume']
        x5['denom'] = x5[['alt_denom', 'volume']].min(axis=1)
        x5['conductance'] = round((x5['boundary']/x5['denom']), 2)

        save_name = '/Users/shreya/Documents/mcl_jsd/conductance/' + name + '_' + str(val) + '_conductance.csv'

        print("Done. Saving to CSV.")
        
        x5.to_csv(save_name, index = None, header=True, encoding='utf-8')


        
        



In [ ]:
# Reading in scp and cluster files separately 

conductance_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/top_nodes_dc.csv")
conductance_data.rename(columns={'scp':'citing', 'ref_sgr':'cited'}, inplace=True)
cluster_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/top_nodes_dc_now_20_cluster_list.csv")

all_data_pd = conductance_data.merge(cluster_data, left_on='citing', right_on='scp', how='inner').rename(columns={'cluster_no':'citing_cluster'}).merge(cluster_data, left_on='cited', right_on='scp', how='inner').rename(columns={'cluster_no':'cited_cluster'})
all_data_pd = all_data_pd[['citing', 'cited', 'citing_cluster', 'cited_cluster']]
conductance_data = all_data_pd

counts_data = cluster_data.groupby('cluster_no', as_index=False).agg('count').rename(columns={'cluster_no':'cluster', 'scp':'cluster_counts'})

x1 = conductance_data[conductance_data.cited_cluster != conductance_data.citing_cluster][['citing', 'citing_cluster']].groupby('citing_cluster', as_index=False).agg('count').rename(columns = {'citing': 'ext_out'})
x2 = conductance_data[conductance_data.cited_cluster != conductance_data.citing_cluster][['cited', 'cited_cluster']].groupby('cited_cluster', as_index=False).agg('count').rename(columns = {'cited': 'ext_in'})
x3 = conductance_data[conductance_data.cited_cluster == conductance_data.citing_cluster][['citing', 'cited_cluster']].groupby('cited_cluster', as_index=False).agg('count').rename(columns = {'citing': 'int_edges'})

x1_clusters = counts_data.merge(x1, left_on = 'cluster', right_on = 'citing_cluster', how = 'left')[['cluster', 'ext_out']]
x1_clusters = x1_clusters.fillna(0)
x2_clusters = counts_data.merge(x2, left_on = 'cluster', right_on = 'cited_cluster', how = 'left')[['cluster', 'ext_in']]
x2_clusters = x2_clusters.fillna(0)
x3_clusters = counts_data.merge(x3, left_on = 'cluster', right_on = 'cited_cluster', how = 'left')[['cluster', 'int_edges', 'cluster_counts']]
x3_clusters = x3_clusters.fillna(0)

x4 = x1_clusters.merge(x2_clusters, left_on='cluster', right_on='cluster', how = 'inner')
x5 = x4.merge(x3_clusters, left_on='cluster', right_on='cluster')
x5['boundary'] = x5['ext_in'] + x5['ext_out']
x5['volume'] = x5['ext_in'] + x5['ext_out'] + 2*x5['int_edges']
x5['two_m'] = conductance_data.shape[0]*2
x5['alt_denom'] = x5['two_m'] - x5['volume']
x5['denom'] = x5[['alt_denom', 'volume']].min(axis=1)
x5['conductance'] = round((x5['boundary']/x5['denom']), 2)
x6 = x5

In [ ]:
import pandas as pd
a_conductance_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/top_nodes_dc_merged_edgelist_clusterlist.csv")
a_conductance_data.head()

In [ ]:
b_conductance_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/test_shuffle_merged_edgelist_clusterlist.csv")
b_conductance_data.head()

In [ ]:
a_conductance_data.equals(b_conductance_data)

In [ ]:
# Reading in scp and cluster files together


conductance_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/test_shuffle_merged_edgelist_clusterlist.csv")
conductance_data.rename(columns={'scp':'citing', 'ref_sgr':'cited', 'scp_cl_no':'citing_cluster', 'ref_sgr_cl_no':'cited_cluster'}, inplace=True)

cluster_data = pd.read_csv("/Users/shreya/Documents/mcl_jsd/consolidated_output/direct_citation/test_shuffle_million_20_clusterlist.csv")
counts_data = cluster_data.groupby('cluster_no', as_index=False).agg('count').rename(columns={'cluster_no':'cluster', 'scp':'cluster_counts'})

x1 = conductance_data[conductance_data.cited_cluster != conductance_data.citing_cluster][['citing', 'citing_cluster']].groupby('citing_cluster', as_index=False).agg('count').rename(columns = {'citing': 'ext_out'})
x2 = conductance_data[conductance_data.cited_cluster != conductance_data.citing_cluster][['cited', 'cited_cluster']].groupby('cited_cluster', as_index=False).agg('count').rename(columns = {'cited': 'ext_in'})
x3 = conductance_data[conductance_data.cited_cluster == conductance_data.citing_cluster][['citing', 'cited_cluster']].groupby('cited_cluster', as_index=False).agg('count').rename(columns = {'citing': 'int_edges'})

x1_clusters = counts_data.merge(x1, left_on = 'cluster', right_on = 'citing_cluster', how = 'left')[['cluster', 'ext_out']]
x1_clusters = x1_clusters.fillna(0)
x2_clusters = counts_data.merge(x2, left_on = 'cluster', right_on = 'cited_cluster', how = 'left')[['cluster', 'ext_in']]
x2_clusters = x2_clusters.fillna(0)
x3_clusters = counts_data.merge(x3, left_on = 'cluster', right_on = 'cited_cluster', how = 'left')[['cluster', 'int_edges', 'cluster_counts']]
x3_clusters = x3_clusters.fillna(0)

x4 = x1_clusters.merge(x2_clusters, left_on='cluster', right_on='cluster', how = 'inner')
x5 = x4.merge(x3_clusters, left_on='cluster', right_on='cluster')
x5['boundary'] = x5['ext_in'] + x5['ext_out']
x5['volume'] = x5['ext_in'] + x5['ext_out'] + 2*x5['int_edges']
x5['two_m'] = conductance_data.shape[0]*2
x5['alt_denom'] = x5['two_m'] - x5['volume']
x5['denom'] = x5[['alt_denom', 'volume']].min(axis=1)
x5['conductance'] = round((x5['boundary']/x5['denom']), 2)

In [ ]:
x5

In [ ]:
x6

In [ ]:



save_name = '/Users/shreya/Documents/consolidated_output/direct_citation/ncf_20_conductance.csv'

print("Done. Saving to CSV.")

x5.to_csv(save_name, index = None, header=True, encoding='utf-8')


        

In [ ]:
x5['conductance'].describe()